In [1]:
import requests

from time import time

# Get Data From API

In [2]:
api_key = "6q7QNKy-oIgk-IMuWisJ-jfN7s6"
time_str = str(time())
url = f"https://mobile.kptncook.com/recipes/de/{time_str}?kptnkey={api_key}"

In [3]:
r = requests.get(url)
r.raise_for_status()

In [4]:
recipes = r.json()

# Turn API Data into Recipe

In [5]:
from typing import Optional, List

from pydantic import BaseModel
from fastapi_camelcase import CamelModel

from mealie.schema.recipe.recipe import Recipe
from mealie.schema.recipe.recipe_step import RecipeStep
from mealie.schema.recipe.recipe_notes import RecipeNote
from mealie.schema.recipe.recipe_nutrition import Nutrition
from mealie.schema.recipe.recipe_ingredient import RecipeIngredient

In [6]:
class LocalizedString(BaseModel):
    en: Optional[str]
    de: Optional[str]
    es: Optional[str]
    fr: Optional[str]
    pt: Optional[str]


class KCNutrition(BaseModel):
    calories: int
    protein: int
    fat: int
    carbohydrate: int


class KCImage(BaseModel):
    name: str
    type: Optional[str]
    url: str


class KptnCook(CamelModel):
    localized_title: LocalizedString
    country: str
    author_comment: LocalizedString
    preparation_time: int
    cooking_time: int
    recipe_nutrition: KCNutrition
    steps_en: List[str]
    steps_de: List[str]
    image_list: List[KCImage]
    
    def get_image_url(self, api_key):
        [image] = [i for i in self.image_list if i.type == "cover"]
        image_url = f"{image.url}?kptnkey={api_key}"
        return image_url
    

def transform_keys(raw):
    result = {}
    mapping = {"stepsEN": "steps_en", "stepsDE": "steps_de"}
    for k, v in raw.items():
        result[mapping.get(k, k)] = raw[k]
    return result

In [7]:
def transform_keys(raw):
    result = {}
    mapping = {"stepsEN": "steps_en", "stepsDE": "steps_de"}
    for k, v in raw.items():
        result[mapping.get(k, k)] = raw[k]
    return result


#recipes[0]
parsed = KptnCook(**transform_keys(recipes[0]))

In [8]:
parsed.get_image_url(api_key)

'https://d2am1qai33sroc.cloudfront.net/image/0ec0a91e-c516-45f4-ab0b-a2fea1fdac2f?kptnkey=6q7QNKy-oIgk-IMuWisJ-jfN7s6'

In [9]:
raw = recipes[0]

In [10]:
list(raw.keys())

['_id',
 'localizedTitle',
 'rtype',
 'authorComment',
 'country',
 'preparationTime',
 'cookingTime',
 'recipeNutrition',
 'stepsEN',
 'stepsDE',
 'authors',
 'retailers',
 'ingredients',
 'imageList',
 'localizedPublishDate',
 'creationDate',
 'updateDate']

In [29]:
# raw["ingredients"]
# raw["imageList"]

[{'name': 'REZ_1631_Cover.jpg',
  'url': 'https://d2am1qai33sroc.cloudfront.net/image/0ec0a91e-c516-45f4-ab0b-a2fea1fdac2f',
  'type': 'cover'},
 {'name': 'REZ_1631_Fav.jpg',
  'url': 'https://d2am1qai33sroc.cloudfront.net/image/067b95ba-b81e-4067-b65c-235031d650f0',
  'type': 'favorite'},
 {'name': 'REZ_1631_01.jpg',
  'url': 'https://d2am1qai33sroc.cloudfront.net/image/55b6c169-3960-420b-9b67-70cb9229b22a',
  'type': 'step'},
 {'name': 'REZ_1631_02.jpg',
  'url': 'https://d2am1qai33sroc.cloudfront.net/image/619e1bf24300008402e010da'},
 {'name': 'REZ_1631_03.jpg',
  'url': 'https://d2am1qai33sroc.cloudfront.net/image/61adf939430000320078026b'},
 {'name': 'REZ_1631_04.jpg',
  'url': 'https://d2am1qai33sroc.cloudfront.net/image/61a0d5254600001b08a6d420'},
 {'name': 'REZ_1631_05.jpg',
  'url': 'https://d2am1qai33sroc.cloudfront.net/image/fd6c2729-5681-4750-b57a-fd807538e022',
  'type': 'step'},
 {'name': 'REZ_1631_06.jpg',
  'url': 'https://d2am1qai33sroc.cloudfront.net/image/8a868119-a3

In [25]:
del(raw["products"])

KeyError: 'products'

In [62]:
api.preparation_time

30

In [11]:
def kptncook_to_mealie(api: KptnCook):
    kwargs = {
        "name": api.localized_title.de,
        "notes": [
            RecipeNote(title="author comment", text=api.author_comment.de),
        ],
        "nutrition": Nutrition(
            calories=api.recipe_nutrition.calories,
            protein_content=api.recipe_nutrition.protein,
            fat_content=api.recipe_nutrition.fat,
            carbohydrate_content=api.recipe_nutrition.carbohydrate,
        ),
        "prep_time": api.preparation_time,
        "cook_time": api.cooking_time,
    }
    return Recipe(**kwargs)

In [12]:
api.recipe_nutrition

NameError: name 'api' is not defined

In [13]:
api = KptnCook(**transform_keys(recipes[0]))
recipe = kptncook_to_mealie(api)

In [14]:
recipe

Recipe(id=None, user_id=UUID('89a5193b-85b2-4787-8f2d-00252189ee51'), group_id=UUID('3f4917fd-f229-4a5e-8825-31b4812e91b5'), name='Linsen-Tomaten-Bolognese', slug='linsen-tomaten-bolognese', image=None, recipe_yield=None, total_time=None, prep_time='30', cook_time='0', perform_time=None, description='', recipe_category=[], tags=[], tools=[], rating=None, org_url=None, recipe_ingredient=[], date_added=None, date_updated=None, recipe_instructions=[], nutrition=Nutrition(calories='756', fat_content='12', protein_content='35', carbohydrate_content='128', fiber_content=None, sodium_content=None, sugar_content=None), settings=RecipeSettings(public=False, show_nutrition=False, show_assets=False, landscape_view=False, disable_comments=True, disable_amount=True, locked=False), assets=[], notes=[RecipeNote(title='author comment', text='Die etwas andere Bolognese')], extras={}, comments=[])

# Save Receipt

In [23]:
from mealie.db.db_setup import SessionLocal
from mealie.repos.all_repositories import get_repositories

from mealie.services.recipe.recipe_data_service import RecipeDataService

In [16]:
session = SessionLocal()
db = get_repositories(session)

In [17]:
user = db.users.get_by_username("admin")

In [18]:
recipe.user_id = user.id
recipe.group_id = user.group_id

In [19]:
recipe_model = db.recipes.create(recipe)

In [20]:
recipe_model.id

UUID('483805f4-db70-4603-9607-475dbb290b53')

In [22]:
api.get_image_url(api_key)

'https://d2am1qai33sroc.cloudfront.net/image/0ec0a91e-c516-45f4-ab0b-a2fea1fdac2f?kptnkey=6q7QNKy-oIgk-IMuWisJ-jfN7s6'

In [24]:
data_service = RecipeDataService(recipe_model.id)

In [25]:
data_service.scrape_image(api.get_image_url(api_key))

[03/24/22 15:19:51] INFO     Image URL: https://d2am1qai33sroc.clou ]8;id=550369;file:///Users/jochen/projects/mealie/mealie/services/recipe/recipe_data_service.py\recipe_data_service.py]8;;\:]8;id=151761;file:///Users/jochen/projects/mealie/mealie/services/recipe/recipe_data_service.py#60\60]8;;\
                             dfront.net/image/0ec0a91e-c516-45f4-ab                          
                             0b-a2fea1fdac2f?kptnkey=6q7QNKy-oIgk-I                          
                             MuWisJ-jfN7s6

[03/24/22 15:19:52] INFO     File Name Suffix .jpg                 ]8;id=948509;file:///Users/jochen/projects/mealie/mealie/services/recipe/recipe_data_service.py\recipe_data_service.py]8;;\:]8;id=158225;file:///Users/jochen/projects/mealie/mealie/services/recipe/recipe_data_service.py#105\105]8;;\

                    INFO     original.jpg minified                              ]8;id=116326;file:///Users/jochen/projects/mealie/mealie/pkgs/img/minify.py\minify.py]8;;\:]8;id=728297;file:///Users/jochen/projects/mealie/mealie/pkgs/img/minify.py#116\116]8;;\

                    INFO     Tiny image saved                                   ]8;id=49962;file:///Users/jochen/projects/mealie/mealie/pkgs/img/minify.py\minify.py]8;;\:]8;id=995251;file:///Users/jochen/projects/mealie/mealie/pkgs/img/minify.py#126\126]8;;\